See fail on project design document näidete jaoks by Kea

In [178]:
import requests 
import os
from datetime import datetime, timedelta
import json
import pandas as pd
import numpy as np
import re
import csv

In [179]:
def get_metadata():
    lines = 50000
    with open(f"arxiv-metadata-oai-snapshot.json", 'r') as f:
        for line in f:
            if lines == 0:
                break
            lines -= 1
            yield line

In [180]:
metadata = get_metadata()

authors_parsed = []
dois = []
arxiv_ids = []
titles = []
versions = []
abstracts = []
authors_str = []
update_dates = []
for paper in metadata:
    metaDict = json.loads(paper)
    arxiv_ids.append(metaDict['id'])
    dois.append(metaDict['doi'])
    titles.append(metaDict['title'])
    authors_parsed.append(metaDict['authors_parsed'])
    authors_str.append(metaDict['authors'])
    versions.append(metaDict['versions'])
    abstracts.append(metaDict['abstract'])
    update_dates.append(metaDict['update_date'])

df = pd.DataFrame({'arxiv_id' : arxiv_ids, 'doi': dois, 'abstract':abstracts, 'title' : titles, 
                'authors_parsed': authors_parsed, 'authors': authors_str, 'versions': versions,
                'update_date': update_dates})
df.head()   

,arxiv_id,doi,abstract,title,authors_parsed,authors,versions,update_date
0,0704.0001,10.1103/PhysRevD.76.013009,A fully differential calculation in perturba...,Calculation of prompt diphoton production cros...,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,...","C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...","[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26
1,0704.0002,None,"We describe a new algorithm, the $(k,\ell)$-...",Sparsity-certifying Graph Decompositions,"[[Streinu, Ileana, ], [Theran, Louis, ]]",Ileana Streinu and Louis Theran,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13
2,0704.0003,None,The evolution of Earth-Moon system is descri...,The evolution of the Earth-Moon system based o...,"[[Pan, Hongjun, ]]",Hongjun Pan,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13
3,0704.0004,None,We show that a determinant of Stirling cycle...,A determinant of Stirling cycle numbers counts...,"[[Callan, David, ]]",David Callan,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23
4,0704.0005,None,In this paper we show how to compute the $\L...,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]",Wael Abu-Shammala and Alberto Torchinsky,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15


In [181]:
mask = np.column_stack([df[col].str.contains(r'withdrawn', na=False) for col in df])
df.loc[mask.any(axis=1)]

,arxiv_id,doi,abstract,title,authors_parsed,authors,versions,update_date
155,0704.0156,10.1029/2007JA012441,This paper has been withdrawn by the authors.\n,Nonlinear force-free coronal magnetic field ex...,"[[He, Han, ], [Wang, Huaning, ]]","Han He, Huaning Wang","[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2015-05-13
212,0704.0213,None,This article has been withdrawn because it h...,Geometric Complexity Theory V: On deciding non...,"[[Narayanan, Ketan D. Mulmuley Hariharan, ]]",Ketan D. Mulmuley Hariharan Narayanan,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2012-09-28
373,0704.0374,None,This paper has been withdrawn due to copyrig...,Gravity-induced electric polarization of matte...,"[[Zon, Boris A., ], [Kretinin, Igor Yu., ]]","Boris A. Zon, Igor Yu. Kretinin","[{'version': 'v1', 'created': 'Tue, 3 Apr 2007...",2007-05-23
479,0704.0480,None,paper withdrawn due to the possible error in...,Enhanced quantum Zeno effect and bunching in t...,"[[Kim, Sungyun, ], [Dudarev, Artem, ], [Brand,...","Sungyun Kim, Artem Dudarev and Joachim Brand","[{'version': 'v1', 'created': 'Wed, 4 Apr 2007...",2008-04-04
1079,0704.1080,None,This paper has been withdrawn by the author ...,The solutions of the N-identical quantum harmo...,"[[Channuie, Phongpichit, ]]",Phongpichit Channuie,"[{'version': 'v1', 'created': 'Mon, 9 Apr 2007...",2011-02-16
...,...,...,...,...,...,...,...,...
47994,0802.0697,None,"This paper has been withdrawn, and is replac...",A new approach to solvability of some elliptic...,"[[Auscher, Pascal, ], [Axelsson, Andreas, ], [...","Pascal Auscher, Andreas Axelsson, Alan McIntosh","[{'version': 'v1', 'created': 'Tue, 5 Feb 2008...",2008-09-29
48050,0802.0753,None,This paper has been withdrawn by the author ...,Using complex networks to model 2-D and 3-D so...,"[[Mooney, Sacha Jon, ], [Korosak, Dean, ]]",Sacha Jon Mooney and Dean Korosak,"[{'version': 'v1', 'created': 'Wed, 6 Feb 2008...",2008-03-15
48399,0802.1102,10.1103/PhysRevB.77.012404,Magneto-structural instability in the ferrom...,Metastability and magnetic memory effect in Ni...,"[[Chatterjee, S., ], [Giri, S., ], [Majumdar, ...","S. Chatterjee, S. Giri, S. Majumdar, and S. K. De","[{'version': 'v1', 'created': 'Fri, 8 Feb 2008...",2009-11-13
48621,0802.1324,None,"This paper has been withdrawn by Chien-Yu, C...",Probe Annihilation process on noncommutative s...,"[[Chen, C. Y., ]]",C. Y. Chen,"[{'version': 'v1', 'created': 'Sun, 10 Feb 200...",2008-08-08


In [182]:
p = re.compile('\s+(This|The) (paper|submission|manuscript) (has been|is being|is) withdrawn')
not_withdrawn = df['abstract'].apply(p.match).isnull()
df = df.loc[not_withdrawn]

In [183]:
df.head()

,arxiv_id,doi,abstract,title,authors_parsed,authors,versions,update_date
0,0704.0001,10.1103/PhysRevD.76.013009,A fully differential calculation in perturba...,Calculation of prompt diphoton production cros...,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,...","C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...","[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26
1,0704.0002,None,"We describe a new algorithm, the $(k,\ell)$-...",Sparsity-certifying Graph Decompositions,"[[Streinu, Ileana, ], [Theran, Louis, ]]",Ileana Streinu and Louis Theran,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13
2,0704.0003,None,The evolution of Earth-Moon system is descri...,The evolution of the Earth-Moon system based o...,"[[Pan, Hongjun, ]]",Hongjun Pan,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13
3,0704.0004,None,We show that a determinant of Stirling cycle...,A determinant of Stirling cycle numbers counts...,"[[Callan, David, ]]",David Callan,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23
4,0704.0005,None,In this paper we show how to compute the $\L...,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]",Wael Abu-Shammala and Alberto Torchinsky,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15


In [184]:
df.drop(['abstract'], axis=1, inplace=True)

In [185]:
df.head()

,arxiv_id,doi,title,authors_parsed,authors,versions,update_date
0,0704.0001,10.1103/PhysRevD.76.013009,Calculation of prompt diphoton production cros...,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,...","C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...","[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26
1,0704.0002,None,Sparsity-certifying Graph Decompositions,"[[Streinu, Ileana, ], [Theran, Louis, ]]",Ileana Streinu and Louis Theran,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13
2,0704.0003,None,The evolution of the Earth-Moon system based o...,"[[Pan, Hongjun, ]]",Hongjun Pan,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13
3,0704.0004,None,A determinant of Stirling cycle numbers counts...,"[[Callan, David, ]]",David Callan,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23
4,0704.0005,None,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]",Wael Abu-Shammala and Alberto Torchinsky,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15


In [186]:
#df.describe() # Siit on näha, et esineb ka duplikaate.

In [187]:
df[df.duplicated(subset=(['title','authors']), keep=False)].sort_values('title') #Vaata duplikaate

,arxiv_id,doi,title,authors_parsed,authors,versions,update_date
14807,0707.1637,None,A partial $A_\infty$-structure on the cohomolo...,"[[Vejdemo-Johansson, Mikael, ]]",Mikael Vejdemo-Johansson,"[{'version': 'v1', 'created': 'Wed, 11 Jul 200...",2010-05-14
45337,0801.3010,None,A partial $A_\infty$-structure on the cohomolo...,"[[Vejdemo-Johansson, Mikael, ]]",Mikael Vejdemo-Johansson,"[{'version': 'v1', 'created': 'Sat, 19 Jan 200...",2010-05-14
26099,0709.3834,None,Back-in-time dynamics of the cluster IE 0657-5...,"[[Nusser, Adi, ]]",Adi Nusser,"[{'version': 'v1', 'created': 'Mon, 24 Sep 200...",2007-10-01
25837,0709.3572,10.1111/j.1365-2966.2007.12716.x,Back-in-time dynamics of the cluster IE 0657-5...,"[[Nusser, Adi, ]]",Adi Nusser,"[{'version': 'v1', 'created': 'Sat, 22 Sep 200...",2009-11-13
40885,0712.2964,10.1111/j.1365-2966.2008.12870.x,Gamma-Ray Emission from PWNe Interacting with ...,"[[Bartko, H., ], [Bednarek, W., ]]","H. Bartko, W. Bednarek","[{'version': 'v1', 'created': 'Tue, 18 Dec 200...",2009-11-13
26835,0709.4570,None,Gamma-Ray Emission from PWNe Interacting with ...,"[[Bartko, H., ], [Bednarek, W., ]]","H. Bartko, W. Bednarek","[{'version': 'v1', 'created': 'Fri, 28 Sep 200...",2013-06-21
22762,0709.0497,None,Hard spectator interactions in B to pi pi at o...,"[[Pilipp, Volker, ]]",Volker Pilipp,"[{'version': 'v1', 'created': 'Tue, 4 Sep 2007...",2007-09-06
25479,0709.3214,10.1016/j.nuclphysb.2007.10.028,Hard spectator interactions in B to pi pi at o...,"[[Pilipp, Volker, ]]",Volker Pilipp,"[{'version': 'v1', 'created': 'Thu, 20 Sep 200...",2008-11-26
38555,0712.0634,10.1007/s00601-008-0312-9,High-precision covariant one-boson-exchange po...,"[[Gross, Franz, ], [Stadler, Alfred, ]]",Franz Gross and Alfred Stadler,"[{'version': 'v1', 'created': 'Wed, 5 Dec 2007...",2009-01-16
1228,0704.1229,10.1016/j.physletb.2007.10.028,High-precision covariant one-boson-exchange po...,"[[Gross, Franz, ], [Stadler, Alfred, ]]",Franz Gross and Alfred Stadler,"[{'version': 'v1', 'created': 'Tue, 10 Apr 200...",2008-11-26


In [188]:
df.loc[22762]['title']

'Hard spectator interactions in B to pi pi at order alpha_s^2'

In [189]:
df.loc[25479]['title']

'Hard spectator interactions in B to pi pi at order alpha_s^2'

In [190]:
df.isnull().sum()

arxiv_id              0
doi               19455
title                 0
authors_parsed        0
authors               0
versions              0
update_date           0
dtype: int64

In [191]:
#Drop duplicates by keeping the one with doi or if doi is missing, a newer update date
df['update_date'] = pd.to_datetime(df['update_date'])

print(f'Ridu enne duplikaatide droppimist: {len(df)}')
df = df.sort_values(by=['doi','update_date']).drop_duplicates(subset=(['title','authors']), keep='last')
print(f'Ridu peale duplikaatide droppimist: {len(df)}')


Ridu enne duplikaatide droppimist: 49870
Ridu peale duplikaatide droppimist: 49860


In [192]:
df[df['authors']== "Adi Nusser"] #Kontroll

,arxiv_id,doi,title,authors_parsed,authors,versions,update_date
30448,0710.3501,10.1016/j.physd.2008.02.018,Boundary-value problems in cosmological dynamics,"[[Nusser, Adi, ]]",Adi Nusser,"[{'version': 'v1', 'created': 'Thu, 18 Oct 200...",2008-11-26
25837,0709.3572,10.1111/j.1365-2966.2007.12716.x,Back-in-time dynamics of the cluster IE 0657-5...,"[[Nusser, Adi, ]]",Adi Nusser,"[{'version': 'v1', 'created': 'Sat, 22 Sep 200...",2009-11-13


In [193]:
df.drop(['authors'], axis=1, inplace=True)
df.drop(['update_date'], axis=1, inplace=True)

In [194]:
# Missing values
df.isnull().sum()

arxiv_id              0
doi               19447
title                 0
authors_parsed        0
versions              0
dtype: int64

In [196]:
len(df)

49860

In [197]:
# Show records where title is only one word
df[~df.title.str.contains(" ")]

,arxiv_id,doi,title,authors_parsed,versions
1378,0704.1379,None,U-max-Statistics,"[[Lao, Wei, ], [Mayer, Michael, ]]","[{'version': 'v1', 'created': 'Wed, 11 Apr 200..."
11607,0706.2921,None,Herwig++,"[[Gigg, Martyn, , IPPP, Durham University], [R...","[{'version': 'v1', 'created': 'Wed, 20 Jun 200..."
28279,0710.1332,None,Polyexponentials,"[[Boyadzhiev, Khristo N., ]]","[{'version': 'v1', 'created': 'Fri, 5 Oct 2007..."
29304,0710.2357,None,Overhang,"[[Paterson, Mike, ], [Zwick, Uri, ]]","[{'version': 'v1', 'created': 'Fri, 12 Oct 200..."
31399,0710.4452,None,f0(1370),"[[Bugg, D. V., , Queen Mary, University of Lon...","[{'version': 'v1', 'created': 'Wed, 24 Oct 200..."
32322,0710.5375,None,Superbosonization,"[[Sommers, Hans-Jürgen, ]]","[{'version': 'v1', 'created': 'Mon, 29 Oct 200..."
38543,0712.0622,None,Filtrations,"[[Coculescu, Delia, ], [Nikeghbali, Ashkan, ]]","[{'version': 'v1', 'created': 'Tue, 4 Dec 2007..."
42645,0801.0318,None,SUSY@LHC.CERN.CH,"[[Spiropulu, Maria, ]]","[{'version': 'v1', 'created': 'Tue, 1 Jan 2008..."
43607,0801.1280,None,LR-algebras,"[[Burde, Dietrich, ], [Dekimpe, Karel, ], [Des...","[{'version': 'v1', 'created': 'Tue, 8 Jan 2008..."
46031,0801.3704,None,$q$-Chaos,"[[Junge, Marius, ], [Lee, Hun Hee, ]]","[{'version': 'v1', 'created': 'Thu, 24 Jan 200..."


In [198]:
# Find records with missing authors
df[df['authors_parsed'].isnull()]

,arxiv_id,doi,title,authors_parsed,versions


In [202]:
from operator import itemgetter
df['latest_version'] = df['versions'].apply(itemgetter(-1))
df.head()

,arxiv_id,doi,title,authors_parsed,versions,latest_version
6819,0705.2817,None,Decoding of scroll codes,"[[Hitching, George H., ], [Johnsen, Trygve, ]]","[{'version': 'v1', 'created': 'Mon, 21 May 200...","{'version': 'v1', 'created': 'Mon, 21 May 2007..."
3976,0704.3977,None,Electronic Properties of Eumelanin Monomers an...,"[[Solórzano, M. M. Falla, ], [Bolivar-Marinez,...","[{'version': 'v1', 'created': 'Mon, 30 Apr 200...","{'version': 'v1', 'created': 'Mon, 30 Apr 2007..."
3979,0704.3980,None,A construction of generalized Harish-Chandra m...,"[[Penkov, Ivan, ], [Zuckerman, Gregg, ]]","[{'version': 'v1', 'created': 'Mon, 30 Apr 200...","{'version': 'v1', 'created': 'Mon, 30 Apr 2007..."
3987,0704.3988,None,Three Applications of the Cuntz Semigroup,"[[Brown, Nathanial P., ], [Toms, Andrew S., ]]","[{'version': 'v1', 'created': 'Mon, 30 Apr 200...","{'version': 'v1', 'created': 'Mon, 30 Apr 2007..."
1378,0704.1379,None,U-max-Statistics,"[[Lao, Wei, ], [Mayer, Michael, ]]","[{'version': 'v1', 'created': 'Wed, 11 Apr 200...","{'version': 'v1', 'created': 'Wed, 11 Apr 2007..."


In [203]:
nrs = [d.get('version') for d in df.latest_version]

In [204]:
df['latest_version_nr'] = nrs
df.head()

,arxiv_id,doi,title,authors_parsed,versions,latest_version,latest_version_nr
6819,0705.2817,None,Decoding of scroll codes,"[[Hitching, George H., ], [Johnsen, Trygve, ]]","[{'version': 'v1', 'created': 'Mon, 21 May 200...","{'version': 'v1', 'created': 'Mon, 21 May 2007...",v1
3976,0704.3977,None,Electronic Properties of Eumelanin Monomers an...,"[[Solórzano, M. M. Falla, ], [Bolivar-Marinez,...","[{'version': 'v1', 'created': 'Mon, 30 Apr 200...","{'version': 'v1', 'created': 'Mon, 30 Apr 2007...",v1
3979,0704.3980,None,A construction of generalized Harish-Chandra m...,"[[Penkov, Ivan, ], [Zuckerman, Gregg, ]]","[{'version': 'v1', 'created': 'Mon, 30 Apr 200...","{'version': 'v1', 'created': 'Mon, 30 Apr 2007...",v1
3987,0704.3988,None,Three Applications of the Cuntz Semigroup,"[[Brown, Nathanial P., ], [Toms, Andrew S., ]]","[{'version': 'v1', 'created': 'Mon, 30 Apr 200...","{'version': 'v1', 'created': 'Mon, 30 Apr 2007...",v1
1378,0704.1379,None,U-max-Statistics,"[[Lao, Wei, ], [Mayer, Michael, ]]","[{'version': 'v1', 'created': 'Wed, 11 Apr 200...","{'version': 'v1', 'created': 'Wed, 11 Apr 2007...",v1


In [205]:
df['latest_version_nr'].value_counts()

v1     31184
v2     12866
v3      4106
v4      1088
v5       365
v6       131
v9        49
v7        43
v8        27
v98        1
Name: latest_version_nr, dtype: int64

In [208]:
df[df['latest_version_nr']=='v98']

,arxiv_id,doi,title,authors_parsed,versions,latest_version,latest_version_nr
9824,0706.1138,None,Some Comments around The Examples against\\ Th...,"[[Oda, Susumu, ]]","[{'version': 'v1', 'created': 'Fri, 8 Jun 2007...","{'version': 'v98', 'created': 'Thu, 8 Sep 2022...",v98


In [211]:
df[df['latest_version_nr']=='v3']

,arxiv_id,doi,title,authors_parsed,versions,latest_version,latest_version_nr
4073,0705.0071,None,A note about the factorization of the angular ...,"[[Alayon-Solarz, Daniel, ]]","[{'version': 'v1', 'created': 'Tue, 1 May 2007...","{'version': 'v3', 'created': 'Fri, 4 May 2007 ...",v3
2571,0704.2572,None,"Homomorphic images of Branch groups, and Serre...","[[Delzant, Thomas, ], [Grigorchuk, Rostislav, ]]","[{'version': 'v1', 'created': 'Thu, 19 Apr 200...","{'version': 'v3', 'created': 'Fri, 4 May 2007 ...",v3
2022,0704.2023,None,Ordered addition of two Lorentz boosts through...,"[[Iyer, Chandru, ], [Prabhu, G. M., ]]","[{'version': 'v1', 'created': 'Mon, 16 Apr 200...","{'version': 'v3', 'created': 'Tue, 8 May 2007 ...",v3
1935,0704.1936,None,Identities for number series and their recipro...,"[[Abrarov, S. M., ], [Abrarov, R. M., ]]","[{'version': 'v1', 'created': 'Mon, 16 Apr 200...","{'version': 'v3', 'created': 'Fri, 20 Apr 2007...",v3
2731,0704.2732,None,Brauer Algebras of Simply Laced Type,"[[Cohen, Arjeh M, ], [Frenk, Bart, ], [Wales, ...","[{'version': 'v1', 'created': 'Fri, 20 Apr 200...","{'version': 'v3', 'created': 'Thu, 26 Apr 2007...",v3
...,...,...,...,...,...,...,...
36724,0711.3832,10.1017/S1474748009000024,Interpr\'etation de l'Arithm\'etique dans cert...,"[[Altınel, Tuna, ], [Muranov, Alexey, ]]","[{'version': 'v1', 'created': 'Mon, 26 Nov 200...","{'version': 'v3', 'created': 'Sat, 12 Sep 2009...",v3
11707,0706.3021,10.2178/bsl/1208358845,Independence property and hyperbolic groups,"[[Jaligot, Eric, ], [Muranov, Alexey, ], [Nema...","[{'version': 'v1', 'created': 'Wed, 20 Jun 200...","{'version': 'v3', 'created': 'Fri, 23 Nov 2007...",v3
41397,0712.3476,10.1007/s00209-013-1187-0,On cyclic fixed points of spectra,"[[Bökstedt, Marcel, ], [Bruner, Robert R., ], ...","[{'version': 'v1', 'created': 'Thu, 20 Dec 200...","{'version': 'v3', 'created': 'Tue, 18 Jun 2013...",v3
41990,0712.4069,10.1007/978-0-8176-4934-0,The Bogomolov multiplier of finite simple groups,"[[Kunyavskii, Boris, ]]","[{'version': 'v1', 'created': 'Tue, 25 Dec 200...","{'version': 'v3', 'created': 'Tue, 2 Aug 2022 ...",v3
